# Homework 4 - Tidy and Process the Billboard Dataset
The Billboard dataset comes with **76 columns** corresponding to the chart position of each song from `x1st.week` through `x76th.week`. This is a classic example of **wide** data that needs to be **melted** (unpivoted) into a long (tidy) format.

### Instructions
1. Follow the instructions on how to setup your Python and Jupyter (or VSCode) environment and cloning or downloading our repository. Instructions can be found in the class notes.
2. Fill the missing pieces of code in the provided notebook.
3. Run the notebook and make sure everything works.


### Dataset Overview
The dataset consists of songs and their weekly chart positions on the Billboard Hot 100. The dataset contains the following columns:
- `year`: The year the song entered the chart.
- `artist`: The artist of the song.
- `track`: The title of the song.
- `time`: The duration of the song.
- `date.entered`: The date the song entered the chart.
- `x1st.week` to `x76th.week`: The chart position of the song for each week.

### Goals

1. **Load** the Billboard dataset from CSV.
2. **Tidy** the data so each row represents one song in one week.
3. **Calculate** the actual date for each week using `date.entered + week * 7 days`.
4. **Split** the data into two tables:
   - A **songs** table with static song information.
   - A **positions** table with `(song_id, week, rank, date)`.
5. **Save** the tidy data to **Feather** format in the same directory with `_tidy` suffix.

### Submission Guidelines

- Submit your completed notebook as a HTML export, or a PDF file.

To export to HTML, if you are on Jupyter, select `File` > `Export Notebook As` > `HTML`.

If you are on VSCode, you can use the `Jupyter: Export to HTML` command.
 - Open the command palette (Ctrl+Shift+P or Cmd+Shift+P on Mac).
     - Search for `Jupyter: Export to HTML`.
     - Save the HTML file to your computer and submit it via Canvas.

---

In [33]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.4 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd

In [2]:
# 1. Load the Billboard dataset
df_bill = pd.read_csv("/Users/rad/Desktop/Useable Ai/Assignments/Final Submission/HW4/billboard.csv")

In [3]:
# Let's check a few columns to see the structure.
df_bill.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 The dataset has columns like:

 - **year**, **artist.inverted**, **track**, **time**, **genre** … (song info)

 - **date.entered**, **date.peaked** … (chart-related dates)

 - **x1st.week** through **x76th.week** … (chart positions over 76 weeks)



 We want to **melt** these weekly columns into a single `week` and `rank` column.

In [4]:
weekly_columns = [col for col in df_bill.columns if 'week' in col and 'x' in col]

In [5]:
melt_df = df_bill.melt(
    id_vars=['year', 'artist.inverted', 'track', 'time', 'genre', 'date.entered', 'date.peaked'],
    value_vars=weekly_columns,
    var_name='week',
    value_name='rank'
)

In [6]:
df_melt1 = melt_df.copy()

In [7]:
melt_df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


 Notice how each row is now **one song** in **one week**. However, the `week` column currently contains strings like `"x1st.week"`, `"x2nd.week"`, etc. Let's clean those up and create a numeric week column.

In [8]:
melt_df['week'] = melt_df['week'].str.extract(r'(\d+)').astype(int)

In [9]:
melt_df

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0
...,...,...,...,...,...,...,...,...,...
24087,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000-08-05,2000-08-05,76,NaN
24088,2000,"Smith, Will",Freakin' It,3:58,Rap,2000-02-12,2000-02-12,76,NaN
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,76,NaN
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,2000-07-01,2000-07-01,76,NaN


 Now, `week = 1, 2, 3, ... 76`. Next, we want to calculate the **exact date** on the chart for each row by adding `week * 7` days to `date.entered`. Create a column named "date" to hold the result. See the expected result in our lecture materials for tidy data.

In [10]:
# Note that after doing that, you should have a new column called date
# Your code here
melt_df['date.entered'] = pd.to_datetime(melt_df['date.entered'])
melt_df['date'] = melt_df['date.entered'] + pd.to_timedelta((melt_df['week'] - 1) * 7, unit='D')

In [11]:
melt_df

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0,1999-10-23
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0,2000-08-05
...,...,...,...,...,...,...,...,...,...,...
24087,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000-08-05,2000-08-05,76,NaN,2002-01-12
24088,2000,"Smith, Will",Freakin' It,3:58,Rap,2000-02-12,2000-02-12,76,NaN,2001-07-21
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,76,NaN,2002-02-09
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,2000-07-01,2000-07-01,76,NaN,2001-12-08


 ### Split into Two Tables



 **Why split?** We often separate the **static** song info (e.g., artist, track, time, genre) from the **weekly** chart performance (week, rank, date).



 - **Songs Table**: Contains unique identifiers for each song plus basic metadata.

 - **Positions Table**: Contains `(song_id, week, rank, date)`, referencing the **song_id** from the songs table.

In [12]:
songs_df = melt_df[['artist.inverted', 'track', 'time', 'genre', 'year', 'date.entered', 'date.peaked']].drop_duplicates().reset_index(drop=True)
songs_df['song_id'] = songs_df.index

positions_df = melt_df.merge(
    songs_df,
    on=['artist.inverted', 'track', 'time', 'genre', 'year', 'date.entered', 'date.peaked'],
    how='left'
)
positions_df = positions_df[['song_id', 'week', 'rank', 'date']]

In [14]:
songs_df

,artist.inverted,track,time,genre,year,date.entered,date.peaked,song_id
0,Destiny's Child,Independent Women Part I,3:38,Rock,2000,2000-09-23,2000-11-18,0
1,Santana,"Maria, Maria",4:18,Rock,2000,2000-02-12,2000-04-08,1
2,Savage Garden,I Knew I Loved You,4:07,Rock,2000,1999-10-23,2000-01-29,2
3,Madonna,Music,3:45,Rock,2000,2000-08-12,2000-09-16,3
4,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000,2000-08-05,2000-10-14,4
...,...,...,...,...,...,...,...,...
312,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000,2000-08-05,2000-08-05,312
313,"Smith, Will",Freakin' It,3:58,Rap,2000,2000-02-12,2000-02-12,313
314,Zombie Nation,Kernkraft 400,3:30,Rock,2000,2000-09-02,2000-09-02,314
315,"Eastsidaz, The",Got Beef,3:58,Rap,2000,2000-07-01,2000-07-01,315


In [15]:
positions_df

,song_id,week,rank,date
0,0,1,78.0,2000-09-23
1,1,1,15.0,2000-02-12
2,2,1,71.0,1999-10-23
3,3,1,41.0,2000-08-12
4,4,1,57.0,2000-08-05
...,...,...,...,...
24087,312,76,NaN,2002-01-12
24088,313,76,NaN,2001-07-21
24089,314,76,NaN,2002-02-09
24090,315,76,NaN,2001-12-08


 Next, we merge this `song_id` back into our `df_tidy` so we can create the positions table.

In [18]:
df_tidy = melt_df.merge(
    songs_df,
    on=['artist.inverted', 'track', 'time', 'genre', 'year', 'date.entered', 'date.peaked'],
    how='left'
)

In [19]:
df_tidy

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date,song_id
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0,2000-09-23,0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0,2000-02-12,1
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0,1999-10-23,2
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0,2000-08-12,3
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0,2000-08-05,4
...,...,...,...,...,...,...,...,...,...,...,...
24087,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000-08-05,2000-08-05,76,NaN,2002-01-12,312
24088,2000,"Smith, Will",Freakin' It,3:58,Rap,2000-02-12,2000-02-12,76,NaN,2001-07-21,313
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,76,NaN,2002-02-09,314
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,2000-07-01,2000-07-01,76,NaN,2001-12-08,315


 ### Create the Positions Table



 We only keep the **relevant columns** for weekly positions: `song_id`, `week`, `rank`, and `date`.

In [20]:
positions_df = df_tidy[['song_id', 'week', 'rank', 'date']]

In [21]:
positions_df

,song_id,week,rank,date
0,0,1,78.0,2000-09-23
1,1,1,15.0,2000-02-12
2,2,1,71.0,1999-10-23
3,3,1,41.0,2000-08-12
4,4,1,57.0,2000-08-05
...,...,...,...,...
24087,312,76,NaN,2002-01-12
24088,313,76,NaN,2001-07-21
24089,314,76,NaN,2002-02-09
24090,315,76,NaN,2001-12-08


## 8.Playing with the data
 Now that we have our data in a tidy format, let's do some analysis.

### Only songs that reached top 10
We can use `query()` to filter the data for songs that reached the top 10 at least once. We will merge this back to the songs table to get the song details.

Get a dataframe with the top 10 songs and their details.

In [22]:
# Your code here
top_10_positions = positions_df.query("rank <= 10")
top_10_song_ids = top_10_positions['song_id'].unique()
top_10_songs_details = songs_df[songs_df['song_id'].isin(top_10_song_ids)]

In [23]:
top_10_songs_details

,artist.inverted,track,time,genre,year,date.entered,date.peaked,song_id
0,Destiny's Child,Independent Women Part I,3:38,Rock,2000,2000-09-23,2000-11-18,0
1,Santana,"Maria, Maria",4:18,Rock,2000,2000-02-12,2000-04-08,1
2,Savage Garden,I Knew I Loved You,4:07,Rock,2000,1999-10-23,2000-01-29,2
3,Madonna,Music,3:45,Rock,2000,2000-08-12,2000-09-16,3
4,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000,2000-08-05,2000-10-14,4
5,Janet,Doesn't Really Matter,4:17,Rock,2000,2000-06-17,2000-08-26,5
6,Destiny's Child,Say My Name,4:31,Rock,2000,1999-12-25,2000-03-18,6
7,"Iglesias, Enrique",Be With You,3:36,Latin,2000,2000-04-01,2000-06-24,7
8,Sisqo,Incomplete,3:52,Rock,2000,2000-06-24,2000-08-12,8
9,Lonestar,Amazed,4:25,Country,2000,1999-06-05,2000-03-04,9


You may want to remove duplicates to get a list of unique songs that reached the top 10. See `df.drop_duplicates()` for more details.

In [24]:
# Your code here
top_10_unique_songs = top_10_songs_details.drop_duplicates(subset='song_id')

In [25]:
top_10_unique_songs

,artist.inverted,track,time,genre,year,date.entered,date.peaked,song_id
0,Destiny's Child,Independent Women Part I,3:38,Rock,2000,2000-09-23,2000-11-18,0
1,Santana,"Maria, Maria",4:18,Rock,2000,2000-02-12,2000-04-08,1
2,Savage Garden,I Knew I Loved You,4:07,Rock,2000,1999-10-23,2000-01-29,2
3,Madonna,Music,3:45,Rock,2000,2000-08-12,2000-09-16,3
4,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000,2000-08-05,2000-10-14,4
5,Janet,Doesn't Really Matter,4:17,Rock,2000,2000-06-17,2000-08-26,5
6,Destiny's Child,Say My Name,4:31,Rock,2000,1999-12-25,2000-03-18,6
7,"Iglesias, Enrique",Be With You,3:36,Latin,2000,2000-04-01,2000-06-24,7
8,Sisqo,Incomplete,3:52,Rock,2000,2000-06-24,2000-08-12,8
9,Lonestar,Amazed,4:25,Country,2000,1999-06-05,2000-03-04,9


### How long did each song stay in the top 10?
Create add to the current dataframe or create a new dataframe with the following columns:
- `song_id` : the song id
- `weeks_in_top_10` : the number of weeks the song was in the top 10

In [26]:
# Your code here
top_10_weeks = positions_df.query("rank <= 10")
weeks_in_top_10_df = top_10_weeks.groupby('song_id').size().reset_index(name='weeks_in_top_10')

In [27]:
weeks_in_top_10_df 

,song_id,weeks_in_top_10
0,0,17
1,1,18
2,2,17
3,3,12
4,4,8
5,5,12
6,6,11
7,7,10
8,8,13
9,9,12


### In which week did each song reach the top 10?
Create or add to a new dataframe with the following columns:
- `week_reached_top_10` : the week in which the song reached the top 10 for the first time

In [28]:
# Your code here
top_10_positions = positions_df.query("rank <= 10")
week_top_10_df = top_10_positions.groupby('song_id')['week'].min().reset_index()
week_top_10_df.rename(columns={'week': 'week_reached_top_10'}, inplace=True)

In [29]:
week_top_10_df

,song_id,week_reached_top_10
0,0,7
1,1,2
2,2,7
3,3,5
4,4,8
5,5,8
6,6,13
7,7,7
8,8,7
9,9,39


 ### 9. Save Tidy Data to Feather



 We want to save:

 - The **tidy** DataFrame (`df_tidy`) to a single file with the suffix `_tidy`.

 - (Optionally) Also save **songs** and **positions** as separate Feather files if needed.

In [31]:
df_tidy.to_csv("billboard_tidy.csv", index=False)

In [34]:
songs_df.to_feather("billboard_songs.feather")
positions_df.to_feather("billboard_positions.feather")